In [46]:
import pandas as pd
import os
current_dir = os.getcwd()
current_dir
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
current_dir
#set current working directory to H:\VScode\March Group\March_Team_Project\
os.chdir("H:\\VScode\\March Group\\March_Team_Project\\")

You set a new current directory


In [47]:
# Issue 16 and Issue 15
# Merge weather data with pollution data, ensure pollution data is filtered for locations and timeperiod.
# Merge on date and location
# Save the merged data to a new csv file
# load pollution data
pollution_data = pd.read_csv('Source_Data\\filtered_pollution.zip')
# count of records in pollution data
print(pollution_data.shape)
# filter pollution data for locations and timeperiod
#pollution_filter = pd.read_csv('Outputs/pollution_data_available.csv')
# set pollution filter to list of states in pollution data
pollution_filter = pollution_data[['State']].drop_duplicates()
print(pollution_filter.shape)
# drop rows with missing values
pollution_data = pollution_data.dropna()
# count of records in pollution filter
print(pollution_data.shape)
# filter pollution_data for location in polution_filter with date between years 2003 and 2016
pollution_data = pollution_data[pollution_data['Date Local'].str.contains('200[0-9]|201[0-6]')]
print(pollution_data.shape)
pollution_data = pollution_data[pollution_data['State'].isin(pollution_filter['State'])]
print(pollution_data.shape)
pollution_data
# drop Unnamed: 0 column
pollution_data = pollution_data.drop(columns=['Unnamed: 0'])
# summarize pollution data
pollution_data_summary = pollution_data.groupby(['State Code',
                                                 'County Code',
                                                 'Site Num',
                                                 'Address',
                                                 'State',
                                                 'County',
                                                 'City',
                                                 'Date Local',
                                                 'Year']).agg(
                                                     {'NO2 Mean': 'mean',
                                                      'NO2 1st Max Value': 'max',
                                                      'NO2 1st Max Hour': 'max',
                                                      'NO2 AQI': 'mean',
                                                      'O3 Mean': 'mean',
                                                      'O3 1st Max Value': 'max',
                                                      'O3 1st Max Hour': 'max',
                                                      'O3 AQI': 'mean',
                                                      'SO2 Mean': 'mean',
                                                      'SO2 1st Max Value': 'max',
                                                      'SO2 1st Max Hour': 'max',
                                                      'SO2 AQI': 'mean',
                                                      'CO Mean': 'mean',
                                                      'CO 1st Max Value': 'max',
                                                      'CO 1st Max Hour': 'max',
                                                      'CO AQI': 'mean'
                                                     }).reset_index()
# summarize pollution data
print(pollution_data_summary.shape)


(878851, 30)
(13, 1)
(219827, 30)
(219827, 30)
(219827, 30)
(213308, 25)


In [48]:
# provide a count of records by year
pollution_data['year'] = pollution_data['Date Local'].str[:4]
pollution_data.groupby('year').size()   


year
2003    18995
2004    18406
2005    19553
2006    20097
2007    22284
2008    23203
2009    19668
2010    18131
2011    19824
2012    19029
2013    20637
dtype: int64

In [49]:
# Create a cleaned version of pollution data where any row with a null value is removed
pollution_data_cleaned = pollution_data.dropna()
print(f"Shape after dropping null values: {pollution_data_cleaned.shape}")

# Format 'City' column as uppercase for consistency
pollution_data_cleaned['City'] = pollution_data_cleaned['City'].str.upper()

# Rename 'Date Local' to 'Date' for better readability and consistency
pollution_data_cleaned.rename(columns={'Date Local': 'Date'}, inplace=True)

# Convert 'Date' column to datetime format for easier date manipulation
pollution_data_cleaned['Date'] = pd.to_datetime(pollution_data_cleaned['Date'])

# Display the first few rows of the cleaned data for verification
pollution_data_cleaned.head()
print(pollution_data_cleaned.shape)



Shape after dropping null values: (219827, 30)
(219827, 30)


In [50]:
# load weather data on day and city
weather_data = pd.read_csv('Not_to_be_shared_to_repo//Us_Weather_Final_10km_V2.zip')
# rename columns to match stations
weather_data = weather_data.rename(columns={'Station_ID':'StationId'})
# convert date to datetime current format yyyymmdd
weather_data['Date'] = weather_data['Date'].astype(str)
weather_data['Date'] = weather_data['Date'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
# drop columns not needed

print(weather_data.shape)
weather_data

(159923, 38)


,Unnamed: 0,StationId,Date,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TSUN,...,WT18,WT19,WT21,WT22,WV03,TMIN,WT07,WV01,WESF,PSUN
0,0,USC00111497,2000-01-01,NaN,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-22.0,NaN,NaN,NaN,NaN
1,1,USC00111497,2000-01-02,NaN,NaN,5.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2,USC00111497,2000-01-03,NaN,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN
3,3,USC00111497,2000-01-04,NaN,NaN,66.0,10.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-39.0,NaN,NaN,NaN,NaN
4,4,USC00111497,2000-01-05,NaN,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159918,159918,USW00093807,2016-12-27,32.0,1423.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN
159919,159919,USW00093807,2016-12-28,21.0,155.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN
159920,159920,USW00093807,2016-12-29,38.0,1317.0,58.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN
159921,159921,USW00093807,2016-12-30,41.0,28.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-27.0,NaN,NaN,NaN,NaN


In [51]:
# load Us_Stations_with_City_10km.csv
stations = pd.read_csv('Source_Data/Us_City_with_Station.csv')
# get siteId from StationId - last 4 characters
stations['SiteId'] = stations['ClosestStation'].str[-8:]
# format StationId as capitals
stations['StationId'] = stations['ClosestStation'].str.upper()
# format city as capitals
stations['City'] = stations['City'].str.upper()
print(stations.shape)
stations
# merge stations with pollution data on address
pollution_data_cleaned = pollution_data_cleaned.merge(stations, on='Address')
print(pollution_data_cleaned.shape)
pollution_data_cleaned


(204, 17)
(219827, 46)


,State Code_x,County Code_x,Site Num_x,Address,State_x,County_x,City_x,Date,NO2 Units,NO2 Mean,...,address,latitude,longitude,formatted_address,ClosestStation,StationDistance,StationLatitude,StationLongitude,SiteId,StationId
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA",US1TXFD0003,1120.503827,34.1176,-99.9406,TXFD0003,US1TXFD0003
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA",US1TXFD0003,1120.503827,34.1176,-99.9406,TXFD0003,US1TXFD0003
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-03,Parts per billion,42.291667,...,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA",US1TXFD0003,1120.503827,34.1176,-99.9406,TXFD0003,US1TXFD0003
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-04,Parts per billion,45.708333,...,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA",US1TXFD0003,1120.503827,34.1176,-99.9406,TXFD0003,US1TXFD0003
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-05,Parts per billion,32.958333,...,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA",US1TXFD0003,1120.503827,34.1176,-99.9406,TXFD0003,US1TXFD0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219822,48,453,14,3724 North Hills Dr,Texas,Travis,AUSTIN,2013-12-27,Parts per billion,15.333333,...,"3724 North Hills Dr, Austin, Texas",30.354768,-97.759575,"3724 N Hills Dr, Austin, TX 78731, USA",US1TXTV0279,0.630147,30.3492,-97.7608,TXTV0279,US1TXTV0279
219823,48,453,14,3724 North Hills Dr,Texas,Travis,AUSTIN,2013-12-28,Parts per billion,12.558333,...,"3724 North Hills Dr, Austin, Texas",30.354768,-97.759575,"3724 N Hills Dr, Austin, TX 78731, USA",US1TXTV0279,0.630147,30.3492,-97.7608,TXTV0279,US1TXTV0279
219824,48,453,14,3724 North Hills Dr,Texas,Travis,AUSTIN,2013-12-29,Parts per billion,4.568182,...,"3724 North Hills Dr, Austin, Texas",30.354768,-97.759575,"3724 N Hills Dr, Austin, TX 78731, USA",US1TXTV0279,0.630147,30.3492,-97.7608,TXTV0279,US1TXTV0279
219825,48,453,14,3724 North Hills Dr,Texas,Travis,AUSTIN,2013-12-30,Parts per billion,8.016667,...,"3724 North Hills Dr, Austin, Texas",30.354768,-97.759575,"3724 N Hills Dr, Austin, TX 78731, USA",US1TXTV0279,0.630147,30.3492,-97.7608,TXTV0279,US1TXTV0279


In [52]:
# # merge weather data with stations on StationId and City
# weather_data = pd.merge(weather_data, stations, on=['StationId'])
print(weather_data.shape)
# set all null values to 0
weather_data = weather_data.fillna(0)
# rename columns Latitude and Longitude to WS_Latitude and WS_Longitude
weather_data = weather_data.rename(columns={'Latitude':'WS_Latitude', 'Longitude':'WS_Longitude', "Elevation": "WS_Elevation", "Name": "WS_Name"})
# filter to only keep 1 record per city per day based on the closest station
print(weather_data.shape)

weather_data

(159923, 38)
(159923, 38)


,Unnamed: 0,StationId,Date,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TSUN,...,WT18,WT19,WT21,WT22,WV03,TMIN,WT07,WV01,WESF,PSUN
0,0,USC00111497,2000-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-22.0,0.0,0.0,0.0,0.0
1,1,USC00111497,2000-01-02,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,USC00111497,2000-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0
3,3,USC00111497,2000-01-04,0.0,0.0,66.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-39.0,0.0,0.0,0.0,0.0
4,4,USC00111497,2000-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159918,159918,USW00093807,2016-12-27,32.0,1423.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
159919,159919,USW00093807,2016-12-28,21.0,155.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0
159920,159920,USW00093807,2016-12-29,38.0,1317.0,58.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0
159921,159921,USW00093807,2016-12-30,41.0,28.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-27.0,0.0,0.0,0.0,0.0


In [53]:
# filter pollution data to have 1 record per city per day using the record with the lowest CityDistance
print(pollution_data_cleaned.shape)
# merge pollution data with weather data on date and location only take first match
merged_data = pd.merge(pollution_data_cleaned, weather_data, on=['Date', 'StationId'], how='inner')
# merged_data = merged_data.sort_values(by=['Date', 'Site', 'StationDistance']).drop_duplicates(subset=['Date', 'Site'], keep='first')
print(merged_data.shape)
merged_data

(219827, 46)
(35487, 82)


,State Code_x,County Code_x,Site Num_x,Address,State_x,County_x,City_x,Date,NO2 Units,NO2 Mean,...,WT18,WT19,WT21,WT22,WV03,TMIN,WT07,WV01,WESF,PSUN
0,4,19,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-01,Parts per billion,22.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,19,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-02,Parts per billion,26.583333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,19,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-03,Parts per billion,15.791667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,19,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-04,Parts per billion,18.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,19,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-05,Parts per billion,17.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35482,6,19,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-27,Parts per billion,30.681818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35483,6,19,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-28,Parts per billion,23.786957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35484,6,19,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-29,Parts per billion,28.730435,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35485,6,19,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-30,Parts per billion,34.823810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# Drop columns that are not needed
merged_data = merged_data.drop(columns=['State Code_x', 'County Code_x', 'City_y', 'County_y', 'State_y', 'State Code_y', 'County Code_y', 'address', 'formatted_address', 'Unnamed: 0', 'O3 Units', 'NO2 Units', 'SO2 Units', 'CO Units'])
# Rename columns for better readability
merged_data.rename(columns={'City_x': 'City', 'County_x': 'County', 'State_x': 'State', 'Site_Num_x': 'Site_Num'}, inplace=True)


In [55]:
# count of records By Date
datecount = merged_data.groupby('Date').size()
datecount

Date
2003-01-01     5
2003-01-02     5
2003-01-03     5
2003-01-04     4
2003-01-05     4
              ..
2013-12-27    21
2013-12-28    18
2013-12-29    19
2013-12-30    20
2013-12-31    10
Length: 4018, dtype: int64

In [56]:
# count unique pollution sites per day
pollution_sites = merged_data.groupby('Date')['Site Num'].nunique()
pollution_sites

KeyError: 'Column not found: Site Num'

In [35]:
# deduplicate merged data to provide 1 row per Date, city, county, state, address, site num, county code, state code, choosing record with lowest CityDistance if multiple records exist
merged_data_sort = merged_data.sort_values(by=['Date','State', 'Address', 'County', 'Site Num', 'City', 'CityDistance']).copy()
print(merged_data_sort.shape)
merged_data_dedupe = merged_data_sort.drop_duplicates(subset=['Date', 'City', 'County', 'State', 'Address', 'Site Num', 'County Code', 'State Code'], keep='first')
print(merged_data_dedupe.shape)


KeyError: 'State'

In [57]:
#  display Percentage of null values in each column
null_values_percentage = merged_data.isnull().sum() / len(merged_data) * 100
null_values_percentage


Site Num_x    0.0
Address       0.0
State         0.0
County        0.0
City          0.0
             ... 
TMIN          0.0
WT07          0.0
WV01          0.0
WESF          0.0
PSUN          0.0
Length: 68, dtype: float64

In [58]:
# Apply AQI classifications to the data
# AQI = 0-50 Good, 51-100 Moderate, 101-150 Unhealthy for Sensitive Groups, 151-200 Unhealthy, 201-300 Very Unhealthy, 301-500 Hazardous
# Create a new column AQI_Range
# Create a new column AQI_Range_Description

def AQI_Range(value):
    if value >= 0 and value <= 50:
        return 'Good'
    if value >= 51 and value <= 100:
        return 'Moderate'
    if value >= 101 and value <= 150:
        return 'Unhealthy for Sensitive Groups'
    if value >= 151 and value <= 200:
        return 'Unhealthy'
    if value >= 201 and value <= 300:
        return 'Very Unhealthy'
    if value >= 301 and value <= 500:
        return 'Hazardous'
    return 'Unknown'

def AQI_Range_Description(value):
    if value >= 0 and value <= 50:
        return 'Air quality is considered satisfactory, and air pollution poses little or no risk.'
    if value >= 51 and value <= 100:
        return 'Air quality is acceptable; however, for some pollutants there may be a moderate health concern for a very small number of people who are unusually sensitive to air pollution.'
    if value >= 101 and value <= 150:
        return 'Members of sensitive groups may experience health effects. The general public is not likely to be affected.'
    if value >= 151 and value <= 200:
        return 'Everyone may begin to experience health effects; members of sensitive groups may experience more serious health effects.'
    if value >= 201 and value <= 300:
        return 'Health alert: everyone may experience more serious health effects.'
    if value >= 301 and value <= 500:
        return 'Health warnings of emergency conditions. The entire population is more likely to be affected.'
    return 'Unknown'

def AQI_Group(value):
    if value >= 0 and value <= 50:
        return 1
    if value >= 51 and value <= 100:
        return 2
    if value >= 101 and value <= 150:
        return 3
    if value >= 151 and value <= 200:
        return 4
    if value >= 201 and value <= 300:
        return 5
    if value >= 301 and value <= 500:
        return 6
    return 'Unknown'

# apply AQI_Range, AQI_Group and AQI_Range_Description to O3_AQI, SO2_AQI, CO_AQI, NO2_AQI
merged_data['O3_AQI_Range'] = merged_data['O3 AQI'].apply(AQI_Range)
merged_data['O3_AQI_Range_Description'] = merged_data['O3 AQI'].apply(AQI_Range_Description)
merged_data['O3_AQI_Group'] = merged_data['O3 AQI'].apply(AQI_Group)
merged_data['SO2_AQI_Range'] = merged_data['SO2 AQI'].apply(AQI_Range)
merged_data['SO2_AQI_Range_Description'] = merged_data['SO2 AQI'].apply(AQI_Range_Description)
merged_data['SO2_AQI_Group'] = merged_data['SO2 AQI'].apply(AQI_Group)
merged_data['CO_AQI_Range'] = merged_data['CO AQI'].apply(AQI_Range)
merged_data['CO_AQI_Range_Description'] = merged_data['CO AQI'].apply(AQI_Range_Description)
merged_data['CO_AQI_Group'] = merged_data['CO AQI'].apply(AQI_Group)
merged_data['NO2_AQI_Range'] = merged_data['NO2 AQI'].apply(AQI_Range)
merged_data['NO2_AQI_Range_Description'] = merged_data['NO2 AQI'].apply(AQI_Range_Description)
merged_data['NO2_AQI_Group'] = merged_data['NO2 AQI'].apply(AQI_Group)

merged_data

,Site Num_x,Address,State,County,City,Date,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,...,O3_AQI_Group,SO2_AQI_Range,SO2_AQI_Range_Description,SO2_AQI_Group,CO_AQI_Range,CO_AQI_Range_Description,CO_AQI_Group,NO2_AQI_Range,NO2_AQI_Range_Description,NO2_AQI_Group
0,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-01,22.250000,44.0,21,42,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
1,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-02,26.583333,50.0,20,47,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
2,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-03,15.791667,30.0,21,28,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
3,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-04,18.200000,41.0,20,39,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
4,1011,"1237 S. BEVERLY , TUCSON",Arizona,Pima,TUCSON,2009-02-05,17.083333,38.0,20,36,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35482,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-27,30.681818,48.6,18,45,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
35483,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-28,23.786957,36.5,19,34,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
35484,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-29,28.730435,53.5,18,50,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
35485,11,"3727 N FIRST ST, FRESNO",California,Fresno,FRESNO,2013-12-30,34.823810,51.5,18,48,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1


In [59]:
# translate Temp from tenths to actual value
merged_data['TMAX'] = merged_data['TMAX'] / 10
merged_data['TMIN'] = merged_data['TMIN'] / 10
merged_data['TAVG'] = merged_data['TAVG'] / 10


In [43]:
# Count records per city - 
# Group by city and count records
city_count = merged_data.groupby(['State', 'City']).size()
expected = 365*10
city_count

State         City         
Arizona       TUCSON            245
California    BURBANK           443
              CALEXICO          164
              COSTA MESA        484
              CUPERTINO         113
              FONTANA           505
              FRESNO           2424
              LONG BEACH        714
              LOS ANGELES      1046
              NOT IN A CITY     286
              RUBIDOUX          943
              SAN JOSE          114
              VICTORVILLE       548
Florida       WINTER PARK      1456
Illinois      NORTHBROOK       3242
Louisiana     BATON ROUGE       960
              KENNER            136
New Jersey    NEWARK           1204
Pennsylvania  BEAVER FALLS     1231
              CHARLEROI        2556
              NEW CASTLE       1267
              NOT IN A CITY      11
Texas         DALLAS           4116
              EL PASO          6339
              HOUSTON          3347
              NOT IN A CITY     842
              SAN ANTONIO       751


In [ ]:
# create extra rows for missing dates in the data, use a fill forward method to fill in missing values
# create a date range from 2003-01-01 to 2016-12-31
date_range = pd.date_range(start='2003-01-01', end='2016-12-31')
# create a dataframe with the date range
date_range_df = pd.DataFrame(date_range, columns=['Date'])
# create a unique list of cities
cities = merged_data['City'].unique()
# create a dataframe with the unique list of cities
cities_df = pd.DataFrame(cities, columns=['City'])
# create a cross join of date_range_df and cities_df
date_city_df = date_range_df.assign(key=1).merge(cities_df.assign(key=1), on

In [60]:
# Save merged to zip file
merged_data.to_csv('Outputs/DashBoardData.zip', index=False)

In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd

def report_imputation_impact(original_df, imputed_df, site_cols):
    exclude_cols = ["Date"] + site_cols
    numeric_cols = original_df.select_dtypes(include="number").columns.difference(exclude_cols)

    summary = []
    for col in numeric_cols:
        total = len(original_df)
        missing_before = original_df[col].isna().sum()
        missing_after = imputed_df[col].isna().sum()
        changed = (original_df[col] != imputed_df[col]) & (~original_df[col].isna())

        summary.append({
            "Column": col,
            "% Missing Before": round(100 * missing_before / total, 2),
            "% Missing After": round(100 * missing_after / total, 2),
            "% Imputed (changed or filled)": round(100 * (missing_before + changed.sum()) / total, 2)
        })

    return pd.DataFrame(summary).sort_values(by="% Imputed (changed or filled)", ascending=False)


# Example usage
site_cols = ["Site Num_y", "latitude", "longitude", "ClosestStation"]
top_columns = ["TAVG", "TMAX", "TMIN", "O3 AQI", "NO2 AQI", "SO2 AQI", "CO AQI", "O3 Mean", "NO2 Mean", "CO Mean"]

subset = filled_df[["Date"] + site_cols + top_columns].copy()
subset["YearMonth"] = subset["Date"].dt.to_period("M")

for col in top_columns:
    subset[col] = subset.groupby(site_cols + ["YearMonth"])[col].transform(lambda x: x.fillna(x.mean()))
    subset[col] = subset.groupby(site_cols)[col].transform(lambda x: x.interpolate(method="linear", limit_direction="both"))

subset.drop(columns=["YearMonth"], inplace=True)

impact_report = report_imputation_impact(filled_df[top_columns], subset[top_columns], site_cols=[])
impact_report
